In [39]:
!pip install langcodes


   ---------------------------------------- 0.0/5.4 MB ? eta -:--:--
   ------- -------------------------------- 1.0/5.4 MB 7.2 MB/s eta 0:00:01
   ------------------------- -------------- 3.4/5.4 MB 13.4 MB/s eta 0:00:01
   ------------------------------- -------- 4.2/5.4 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------- 5.4/5.4 MB 8.4 MB/s eta 0:00:00

   ------------- -------------------------- 1/3 [language-data]
   ------------- -------------------------- 1/3 [language-data]
   ------------- -------------------------- 1/3 [language-data]
   ------------- -------------------------- 1/3 [language-data]
   ------------- -------------------------- 1/3 [language-data]
   -------------------------- ------------- 2/3 [langcodes]
   -------------------------- ------------- 2/3 [langcodes]
   ---------------------------------------- 3/3 [langcodes]



In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
from langdetect import detect
import nltk
import re
import langcodes
nltk.download('punkt_tab', quiet=True, download_dir="../venv/nltk_data")
nltk.data.path.append("../venv/nltk_data")

def code_to_language_name(code: str) -> str:
    try:
        return langcodes.get(code).display_name().lower()
    except:
        return None

In [76]:
def split_sentence(text):
    try:
        lang = detect(text)[:2]
    except:
        return [text]

    if lang in [
        'cs',
        'nl',
        'en',
        'fr', 
        'de',
        'it',
        'pt',
        'ru',
        'es',
        'tr',
    ]:
        return nltk.sent_tokenize(text, language=code_to_language_name(lang))

    elif lang == 'vi':
        return [s.strip() for s in text.replace("!", ".").replace("?", ".").split(".") if s.strip()]

    elif lang == 'zh': # Chinese
        import re
        return re.split(r'(。|！|\!|\.|？|\?)', text)

    elif lang == 'ar': # Arabic
        return [s.strip() for s in text.replace("؟", ".").replace("!", ".").split(".") if s.strip()]

    else: # Defautl
        return [s.strip() for s in text.replace("!", ".").replace("?", ".").split(".") if s.strip()]

In [61]:
import pickle
import os

with open(os.path.join("..", "Dataset", "abstractive", "abstractive.pk"), "rb") as f:
    data = pickle.load(f)

df = data
df.head()

,document,summary,source
0,"When learning how to solve a Rubik’s cube, it ...",Solve a cube from many different sides rather ...,English
1,"Die richtige Methode anzuwenden, um ein Baby z...",Merke dir das Akronym BACK. Vermeide den „Haus...,German
2,يتيح كل موقع من مواقع التواصل الاجتماعي خيارات...,احذفه أو الغِ صداقته أو متابعته على مواقع التو...,Arabic
3,如果你的礼物接收者居住在寒冷的地方这将留给他一个持久的回忆并且很实用。如果不知道如何织，可以...,织一条毛绒绒暖和的围巾。 个性化的衣服。 给猫或马缝一件衣服。,Chinese
4,Utiliza una base de cobertura ligera que luzca...,Utiliza base de maquillaje con moderación. Enf...,Spanish


In [74]:
del dataset['Indonesian']

In [75]:
list(dataset.keys())

['Arabic',
 'Chinese',
 'Czech',
 'Dutch',
 'English',
 'French',
 'German',
 'Italian',
 'Portuguese',
 'Russian',
 'Spanish',
 'Turkish',
 'Vietnamese']

In [77]:
dataset_sentences = dict()
for key, value in dataset.items():
    dataset_sentences[key] = value.map(split_sentence)

list(dataset_sentences.keys())

['Arabic',
 'Chinese',
 'Czech',
 'Dutch',
 'English',
 'French',
 'German',
 'Italian',
 'Portuguese',
 'Russian',
 'Spanish',
 'Turkish',
 'Vietnamese']

In [78]:
with open("../Dataset/extractive/extractive.pk", "wb") as f:
    pickle.dump(dataset_sentences, f)